In [1]:
import winsound as ws
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import cv2
from sklearn.cluster import KMeans

***
<h1>Cleaning and clustering book's data</h1>

In [2]:
alldata = pd.read_csv("dataset\\finals\\allbooks.csv")

In [3]:
#make ratings in percentage
alldata["ratings_1"] = alldata["ratings_1"]/alldata["work_ratings_count"]
alldata["ratings_2"] = alldata["ratings_2"]/alldata["work_ratings_count"]
alldata["ratings_3"] = alldata["ratings_3"]/alldata["work_ratings_count"]
alldata["ratings_4"] = alldata["ratings_4"]/alldata["work_ratings_count"]
alldata["ratings_5"] = alldata["ratings_5"]/alldata["work_ratings_count"]

#change price coloumn name 
alldata["price"] = alldata["price(USD)"]
alldata = alldata.drop( axis = 1, columns = ["price(USD)"])

In [4]:
alldata = alldata.drop(axis = 1, columns=["Unnamed: 0"])

In [5]:
#selecting some data
alldata = alldata[["id", "book_id", "authors", "title", "original_publication_year", "average_rating","work_ratings_count", "ratings_1", "ratings_2", "ratings_3", "ratings_4", "ratings_5", "color_name", "price"]]

In [6]:
alldata = alldata.sort_values(["id"])
alldata

,id,book_id,authors,title,original_publication_year,average_rating,work_ratings_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,color_name,price
0,1,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2008.0,4.34,4942365,0.013499,0.025886,0.113325,0.299716,0.547575,Black,6.83
1,2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997.0,4.44,4800065,0.015730,0.021182,0.094795,0.240896,0.627396,White,49.78
2,3,41865,Stephenie Meyer,"Twilight (Twilight, #1)",2005.0,3.57,3916824,0.116470,0.111519,0.202541,0.223414,0.346056,Black,16.99
3,4,2657,Harper Lee,To Kill a Mockingbird,1960.0,4.25,3340896,0.018087,0.035145,0.133747,0.299905,0.513116,Blue,16.99
4,5,4671,F. Scott Fitzgerald,The Great Gatsby,1925.0,3.89,2773745,0.031090,0.071247,0.218534,0.337454,0.341675,Brown,17.00
5,6,11870085,John Green,The Fault in Our Stars,2012.0,4.26,2478609,0.019363,0.037409,0.132151,0.281800,0.529277,Cyan,19.99
6,7,5907,J.R.R. Tolkien,The Hobbit,1937.0,4.25,2196809,0.020950,0.034953,0.131395,0.303001,0.509702,Black,10.99
7,8,5107,J.D. Salinger,The Catcher in the Rye,1951.0,3.79,2120637,0.051580,0.087483,0.214578,0.311942,0.334416,Orange,15.00
8,9,960,Dan Brown,"Angels & Demons (Robert Langdon, #1)",2000.0,3.85,2078754,0.037446,0.070109,0.220531,0.344711,0.327203,Blue,12.37
9,10,1885,Jane Austen,Pride and Prejudice,1813.0,4.24,2191465,0.024960,0.039464,0.129982,0.278241,0.527352,White,7.52


In [8]:
#discrtize data
palettelabels = ["Black", "White", "Red", "Green", "Blue", "Yellow", "Violet", "Cyan", "Brown", "Orange" ]
def setcolorvalue(x):
    for i in range(10):
        if(x == palettelabels[i]):
            return i
    return -1
alldata["color"] = alldata["color_name"].map(setcolorvalue)


#-----
def pricetag(x):
    if(x < 5):#verychep
        return 1
    elif(x < 15): #cheap
        return 2
    elif(x < 30):#normal
        return 3
    elif(x < 50):#expansive
        return 4
    else:#gold book
        return 5
    
alldata["pricetag"] = alldata["price"].map(pricetag)

In [9]:
gender = pd.read_csv("dataset\\new\\elaboratedtagsselection.csv")[["tag_id", "translation"]]
gender.head()

,tag_id,translation
0,33114,young adult
1,11743,fiction
2,10064,dystopian
3,10059,dystopia
4,11305,fantasy


In [10]:
connector = pd.read_csv("dataset\\book_tags.csv")
connector.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [11]:
#set genders
iddata = alldata["book_id"]
gendertagid = set(gender["tag_id"].values.tolist())

listgenders = []
listgendersnames = []
listgendersclass = []
for idid in iddata:
    tagperbook = set(connector[connector.goodreads_book_id == idid][0:10]["tag_id"].values.tolist())
    taglist = list(tagperbook.intersection(gendertagid))
    
    if(taglist):#if is not empty
        listgenders.append(taglist[0])   
        listgendersnames.append(gender[gender.tag_id == taglist[0]]["translation"].values[0])
        listgendersclass.append(gender[gender.tag_id == taglist[0]].index[0])
    else:
        listgenders.append(None)
        listgendersnames.append(None)
        listgendersclass.append(-1)
        
alldata["genderid"] = listgenders
alldata["gendernames"] = listgendersnames
alldata["genderclass"] = listgendersclass

alldata = alldata.dropna(axis=0, subset=["genderid"])

In [12]:
#replacing with main author
auth= alldata["authors"].values.tolist()
newauth = []
for a in auth:
    newauth.append(a.split(",")[0])
alldata = alldata.assign(authors = newauth)

#authors discratization
freqauthors = alldata["authors"].value_counts().index.values.tolist()
authors = alldata["authors"]
listauthors = []
for author in authors:
    listauthors.append(freqauthors.index(author))
alldata = alldata.assign(discreteauthors = listauthors)

alldata[alldata.ratings_2>0.1]

,id,book_id,authors,title,original_publication_year,average_rating,work_ratings_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,color_name,price,color,pricetag,genderid,gendernames,genderclass,discreteauthors
2,3,41865,Stephenie Meyer,"Twilight (Twilight, #1)",2005.0,3.57,3916824,0.116470,0.111519,0.202541,0.223414,0.346056,Black,16.99,0,3,31745.0,vampires,6,111
33,34,10818853,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",2011.0,3.67,1436818,0.115154,0.105993,0.175516,0.205298,0.398039,Black,NaN,0,5,26138.0,romance,11,372
39,40,19501,Elizabeth Gilbert,"Eat, Pray, Love",2006.0,3.51,1206597,0.083187,0.123943,0.257097,0.275312,0.260461,White,7.85,1,2,19733.0,memoir,20,555
48,49,49041,Stephenie Meyer,"New Moon (Twilight, #2)",2006.0,3.52,1199000,0.085769,0.133995,0.245377,0.242379,0.292480,Black,19.99,0,3,31745.0,vampires,6,111
51,52,428263,Stephenie Meyer,"Eclipse (Twilight, #3)",2007.0,3.69,1176642,0.070620,0.105634,0.221616,0.262916,0.339214,Black,19.99,0,3,31745.0,vampires,6,111
55,56,1162543,Stephenie Meyer,"Breaking Dawn (Twilight, #4)",2008.0,3.70,1107709,0.091174,0.103721,0.192652,0.241675,0.370778,Black,22.99,0,3,26785.0,sci fi fantasy,23,111
104,105,18710190,Veronica Roth,"Allegiant (Divergent, #3)",2013.0,3.63,663226,0.047685,0.118159,0.265462,0.290678,0.278017,Black,25.81,0,3,10059.0,dystopia,3,120
107,108,9416,Sophie Kinsella,"Confessions of a Shopaholic (Shopaholic, #1)",2000.0,3.61,558004,0.055026,0.102058,0.280708,0.299150,0.263057,Violet,0.01,6,1,1659.0,adult fiction,37,83
121,122,37442,Gregory Maguire,Wicked: The Life and Times of the Wicked Witch...,1995.0,3.52,525498,0.077644,0.125281,0.251306,0.292262,0.253506,Yellow,16.00,5,3,26785.0,sci fi fantasy,23,283
137,138,12296,Nathaniel Hawthorne,The Scarlet Letter,1850.0,3.37,559246,0.062250,0.143844,0.326692,0.295181,0.172033,Black,0.97,0,1,14467.0,historical,21,1000


In [13]:
alldata.to_csv("dataset\\finals\\complissivolibricongenere.csv")

***
<h1>Cleaning and coherance with booksdata</h1>

In [15]:
userbook = pd.read_csv("dataset\\ratings.csv")
userbook.head()
#clean the userbook
bookidlist = userbook.drop_duplicates(subset=["book_id"], keep="first")["book_id"].values.tolist()
for bid in bookidlist:
    if(alldata[alldata.id == bid].empty):
        userbook = userbook[userbook.book_id != bid]
userbook.to_csv("dataset\\finals\\ratings_cleaned.csv")